In [15]:
# Install necessary libraries
# !pip install boto3 sagemaker pandas scikit-learn

# Import necessary libraries
import boto3
import sagemaker
from sagemaker import get_execution_role
import time

# Define parameters
role = 'arn:aws:iam::728939754004:role/service-role/AmazonSageMaker-ExecutionRole-20230821T133341'  # Your SageMaker execution role ARN
region = 'us-east-1'
s3_bucket = 'justin-inferences'  # Your S3 bucket for model artifacts
model_artifact_path = f's3://{s3_bucket}/model/model.pkl'
endpoint_name = 'simple-linear-regression-endpoint'
endpoint_config_name = 'simple-linear-regression-endpoint-config'

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Initialize SageMaker client
sm_client = boto3.client('sagemaker', region_name=region)

# Function to delete existing endpoint configuration
def delete_endpoint_config(config_name):
    try:
        sm_client.describe_endpoint_config(EndpointConfigName=config_name)
        print(f"Endpoint configuration {config_name} already exists. Deleting it...")
        sm_client.delete_endpoint_config(EndpointConfigName=config_name)
        print(f"Deleted endpoint configuration {config_name}.")
    except sm_client.exceptions.ClientError as e:
        if 'ValidationException' in str(e):
            print(f"Endpoint configuration {config_name} does not exist. Proceeding to create a new one.")
        else:
            raise e

# Function to delete existing endpoint
def delete_endpoint(endpoint_name):
    try:
        sm_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint {endpoint_name} already exists. Deleting it...")
        sm_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"Deleted endpoint {endpoint_name}.")
        # Wait for the endpoint to be deleted
        waiter = sm_client.get_waiter('endpoint_deleted')
        waiter.wait(EndpointName=endpoint_name)
        print(f"Endpoint {endpoint_name} deleted.")
    except sm_client.exceptions.ClientError as e:
        if 'ValidationException' in str(e):
            print(f"Endpoint {endpoint_name} does not exist. Proceeding to create a new one.")
        else:
            raise e

# Delete existing endpoint configuration if it exists
delete_endpoint_config(endpoint_config_name)

# Delete existing endpoint if it exists
delete_endpoint(endpoint_name)


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Admin\AppData\Local\sagemaker\sagemaker\config.yaml
Endpoint configuration simple-linear-regression-endpoint-config does not exist. Proceeding to create a new one.
Endpoint simple-linear-regression-endpoint does not exist. Proceeding to create a new one.


In [14]:
# Ensure endpoint configuration is deleted before proceeding
while True:
    try:
        sm_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
        print(f"Waiting for endpoint configuration {endpoint_config_name} to be deleted...")
        time.sleep(10)
    except sm_client.exceptions.ClientError as e:
        if 'ValidationException' in str(e):
            print(f"Endpoint configuration {endpoint_config_name} has been deleted.")
            break
        else:
            raise e

Endpoint configuration simple-linear-regression-endpoint-config has been deleted.


In [11]:
# Create a SageMaker model
model = sagemaker.model.Model(
    model_data=model_artifact_path,
    image_uri=sagemaker.image_uris.retrieve('linear-learner', region),
    role=role,
    sagemaker_session=sagemaker_session
)


In [12]:
# Deploy the model to an endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name=endpoint_name
)

print(f"Endpoint {endpoint_name} created successfully.")

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:728939754004:endpoint-config/simple-linear-regression-endpoint".

In [16]:
# Install necessary libraries
# !pip install boto3 sagemaker pandas scikit-learn

# Import necessary libraries
import boto3
import sagemaker
from sagemaker import get_execution_role
import time

# Define parameters
role = 'arn:aws:iam::728939754004:role/service-role/AmazonSageMaker-ExecutionRole-20230821T133341'  # Your SageMaker execution role ARN
region = 'us-east-1'
s3_bucket = 'justin-inferences'  # Your S3 bucket for model artifacts
model_artifact_path = f's3://{s3_bucket}/model/model.pkl'
endpoint_name = 'simple-linear-regression-endpoint'
endpoint_config_name = 'simple-linear-regression-endpoint-config'

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Initialize SageMaker client
sm_client = boto3.client('sagemaker', region_name=region)

# Function to delete existing endpoint configuration
def delete_endpoint_config(config_name):
    try:
        sm_client.describe_endpoint_config(EndpointConfigName=config_name)
        print(f"Endpoint configuration {config_name} already exists. Deleting it...")
        sm_client.delete_endpoint_config(EndpointConfigName=config_name)
        print(f"Deleted endpoint configuration {config_name}.")
    except sm_client.exceptions.ClientError as e:
        if 'ValidationException' in str(e):
            print(f"Endpoint configuration {config_name} does not exist. Proceeding to create a new one.")
        else:
            raise e

# Function to delete existing endpoint
def delete_endpoint(endpoint_name):
    try:
        sm_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint {endpoint_name} already exists. Deleting it...")
        sm_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"Deleted endpoint {endpoint_name}.")
        # Wait for the endpoint to be deleted
        waiter = sm_client.get_waiter('endpoint_deleted')
        waiter.wait(EndpointName=endpoint_name)
        print(f"Endpoint {endpoint_name} deleted.")
    except sm_client.exceptions.ClientError as e:
        if 'ValidationException' in str(e):
            print(f"Endpoint {endpoint_name} does not exist. Proceeding to create a new one.")
        else:
            raise e

# Delete existing endpoint configuration if it exists
delete_endpoint_config(endpoint_config_name)

# Ensure endpoint configuration is deleted before proceeding
while True:
    try:
        sm_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
        print(f"Waiting for endpoint configuration {endpoint_config_name} to be deleted...")
        time.sleep(10)
    except sm_client.exceptions.ClientError as e:
        if 'ValidationException' in str(e):
            print(f"Endpoint configuration {endpoint_config_name} has been deleted.")
            break
        else:
            raise e

# Delete existing endpoint if it exists
delete_endpoint(endpoint_name)




sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Admin\AppData\Local\sagemaker\sagemaker\config.yaml
Endpoint configuration simple-linear-regression-endpoint-config does not exist. Proceeding to create a new one.
Endpoint configuration simple-linear-regression-endpoint-config has been deleted.
Endpoint simple-linear-regression-endpoint does not exist. Proceeding to create a new one.


In [28]:
# Install necessary libraries
# !pip install boto3 sagemaker pandas scikit-learn

# Import necessary libraries
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import Predictor

# Define parameters
role = 'arn:aws:iam::728939754004:role/service-role/AmazonSageMaker-ExecutionRole-20230821T133341'  # Your SageMaker execution role ARN
region = 'us-east-1'
s3_bucket = 'justin-inferences'  # Your S3 bucket for model artifacts
model_artifact_path = f's3://{s3_bucket}/model/model.pkl'
endpoint_name = 'simple-linear-regression-endpoint'
instance_type = 'ml.m5.large'
endpoint_config_name = 'simple-linear-regression-endpoint-config'

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Initialize SageMaker client
sm_client = boto3.client('sagemaker', region_name=region)

# Function to delete existing endpoint configuration
def delete_endpoint_config(config_name):
    try:
        sm_client.describe_endpoint_config(EndpointConfigName=config_name)
        print(f"Endpoint configuration {config_name} already exists. Deleting it...")
        sm_client.delete_endpoint_config(EndpointConfigName=config_name)
        print(f"Deleted endpoint configuration {config_name}.")
    except sm_client.exceptions.ClientError as e:
        if e.response['Error']['Code'] == 'ValidationException':
            print(f"Endpoint configuration {config_name} does not exist. Proceeding to create a new one.")
        else:
            raise e

# Function to create or update an endpoint
def create_or_update_endpoint(model, endpoint_name):
    try:
        # Try to describe the endpoint
        response = sm_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint {endpoint_name} already exists. Updating it...")
        
        # Update the endpoint with a new model
        model.deploy(
            initial_instance_count=1,
            instance_type=instance_type,
            endpoint_name=endpoint_name,
            update_endpoint=True
        )
        print(f"Endpoint {endpoint_name} updated successfully.")
    except sm_client.exceptions.ClientError as e:
        if e.response['Error']['Code'] == 'ValidationException':
            # If the endpoint does not exist, create it
            print(f"Endpoint {endpoint_name} does not exist. Creating it...")
            model.deploy(
                initial_instance_count=1,
                instance_type=instance_type,
                endpoint_name=endpoint_name
            )
            print(f"Endpoint {endpoint_name} created successfully.")
        else:
            print(f"Failed to create or update endpoint {endpoint_name}: {str(e)}")

# Delete existing endpoint configuration if it exists
delete_endpoint_config(endpoint_config_name)

# Create a SageMaker model
model = sagemaker.model.Model(
    model_data=model_artifact_path,
    image_uri=sagemaker.image_uris.retrieve('linear-learner', region),
    role=role,
    sagemaker_session=sagemaker_session
)

# Create or update the endpoint
create_or_update_endpoint(model, endpoint_name)


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Admin\AppData\Local\sagemaker\sagemaker\config.yaml
Endpoint configuration simple-linear-regression-endpoint-config does not exist. Proceeding to create a new one.
Endpoint simple-linear-regression-endpoint does not exist. Creating it...


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:728939754004:endpoint-config/simple-linear-regression-endpoint".